In [13]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [14]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')
# データフレームを表示して確認
df.head()

,アンケート
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...


In [18]:
# 自由記述アンケートをLLMで要約する関数
def summarize_survey(text):
    # 2. データをLLM用にテキスト形式に変換
    # プロンプトの作成
    prompt_text = f"以下の自由記述のアンケートの要約を作成してください。\nアンケート: {text}"

    # 3. OpenAI APIの呼び出し
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            # max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # 要約結果を取得
        summarized_survey = response.choices[0].message.content.strip()
        return summarized_survey
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [16]:
# 結果をファイルに保存する関数
def save_result_to_file(result, file_path="自由記述アンケート要約結果.txt"):
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(result)

In [20]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

survey_text = ""

for index, row in df.iterrows():
    survey_text += str(index) + ". " + row['アンケート'] + '\n'


summarized_text = summarize_survey(survey_text)

save_result_to_file(summarized_text)

print("txtファイルに要約結果を保存しました。")
print(summarized_text)


処理を開始します。
txtファイルに要約結果を保存しました。
アンケートの要約:

多くの回答者がシャンプーの使用感や香りに満足しており、特に髪のしっとり感や指通りの良さが評価されています。香りについては、甘めの香りが好みが分かれるものの、リラックスできると感じる人が多いです。また、洗い上がりが軽やかで頭皮のすっきり感も好評です。ボトルデザインがシンプルでおしゃれだという意見もあり、バスルームの雰囲気を良くする点が評価されています。敏感肌の方からは、刺激が少なく使いやすいという声もあり、環境に配慮したリサイクル素材のボトルも好感を持たれています。全体的にリピート意向が強く、家族での使用も広がっているようです。
